<a href="https://colab.research.google.com/github/88FaHaD/ML/blob/main/mobilenetimproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models



Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3075/3075), done.
remote: Total 4084 (delta 1189), reused 2900 (delta 949), pack-reused 0
Receiving objects: 100% (4084/4084), 44.61 MiB | 26.97 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.


In [4]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━

In [6]:
! pip install kaggle

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! mkdir ~/.kaggle

In [9]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download gpiosenka/100-bird-species

 99% 1.95G/1.96G [00:21<00:00, 127MB/s]
100% 1.96G/1.96G [00:21<00:00, 96.8MB/s]


In [13]:
import zipfile

zip_file_path = "/content/100-bird-species.zip"

extract_to_directory = "/content/bird_species_data"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (160, 160)
BATCH_SIZE = 32

train_dir = '/content/bird_species_data/train'
valid_dir = '/content/bird_species_data/valid'
test_dir=   '/content/bird_species_data/test'

In [16]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Data augmentation for validation and test data (only rescaling)
valid_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)

# Load and augment training data
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')

# Load and augment validation data
valid_data = valid_datagen.flow_from_directory(valid_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical')

# Load and augment test data
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical')


Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [17]:
!pip install tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844486 sha256=5938a438bc46a3e475d4465aecff781db8b2f3f9c4b323258910fd87acdeca22
  Stored in directory: /root/.cache/pip/wheels/8a/55/68/c084bc2cd93c41fd8f7e2ef9e6bbcb2c35a3e4b49e42044d02
Successfully built tensorflow-object-detection-api


In [19]:
from tensorflow.keras import layers, Model

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, Model


base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(525, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
BATCH_SIZE = 32

history = model.fit(train_data,
                    epochs=10,
                    steps_per_epoch=len(train_data)//BATCH_SIZE,
                    validation_data=valid_data,
                    validation_steps=len(valid_data)//BATCH_SIZE)

test_loss, test_acc = model.evaluate(test_data, batch_size=BATCH_SIZE)
print("Test Accuracy:", test_acc)


In [ ]:
model.evaluate(test_data)

In [ ]:
import pandas as pd


bird_csv = "/content/bird_species_data/birds.csv"
birds_df = pd.read_csv(bird_csv)


class_labels = birds_df["labels"].tolist()


class_names = [str(label) for label in class_labels]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_data = test_data_gen.flow_from_directory(test_dir, target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE, class_mode="categorical")


In [ ]:
batch_images, batch_labels = next(test_data)

In [ ]:
num_images_to_predict = 5
selected_images = batch_images[:num_images_to_predict]

In [ ]:
def make_predictions(images):
    return model.predict(images)

In [ ]:
predictions = model.predict(selected_images)

In [ ]:
for i, pred in enumerate(predictions):
    predicted_class_index = np.argmax(pred)
    predicted_class = class_names[predicted_class_index]
    confidence = pred[predicted_class_index] * 100

    plt.imshow(selected_images[i])
    plt.title(f"Predicted class: {predicted_class}, Confidence: {confidence:.2f}%")
    plt.axis('off')
    plt.show()